In [8]:
import pandas as pd
import numpy as np 
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
import os

In [9]:


def read_input( train_path,test_path ):
    train = pd.read_csv(train_path)
    #print(train.head())
    print('df.shape :',train.shape)
    test = pd.read_csv(test_path)
    #print(test.head())
    print('df.shape :',test.shape)
    return train,test

def spilt_data( train,test ):
    value_columns = []
    for i in range(1,50):
        string = 'value_'+str(i)
        value_columns.append(string)   

    train_y = train[value_columns]
    test_y = test[value_columns]

    for col in value_columns:
        train.pop(col)
        test.pop(col)

    train_x = train
    test_x = test
    
    return train_y,test_y,train_x,test_x

In [56]:
def output(outputfile,predlist):
    np.savetxt(outputfile, predlist, delimiter=",")
    
def getscore(test_y,predict_test):
    #compute Accuracy
    test_y[test_y ==0] =1e-10
    tmp = abs( test_y - predict_test )/abs( test_y )
    #print(tmp)
    #print(type(tmp))
    #print("-----------------------------------------------------------")
    tmp[tmp >1] =1
    #print(tmp**2)
    square = tmp**2
    #print(square.shape[1])
    squ_sum = square.sum()
    #print(sum1)
    n = square.shape[0] * square.shape[1]
    #print(n)
    score = 100 - 100 * (np.sqrt(squ_sum/n))
    print("accuracy:" + str(score))
    
    return score,squ_sum

In [57]:
# xgboost模型


import xgboost as xgb
def xgb_train( train_x,train_y,test_x,test_y,testout_path ,test_file_name):
    xgb_len = len(test_y.T)
    maxdepth = [6]#30
    lr = [0.05]#0.07
    nest = [500]#150

    #train all case
    for md in maxdepth:
        for l in lr:
            for n in nest:
                predict_test = []
                print( "md:" + str(md) )
                print( "lr:" + str(l) )
                print( "nest:" + str(n) )

                for i in range(xgb_len):
                    #print("value" +  str(i) +"  training" )
                    params = {
                    "max_depth":int(md),
                    "learning_rate":float(l),
                    "n_estimators":int(n),
                    "booster":"gbtree",
                    #"subsample":0.9,
                    #"colsample_bytree":0.8
                    }

                    clf = xgb.XGBRegressor(**params)
                    eval_set = [(test_x, test_y)]

                    #clf.fit(train_x, train_y.T[i], eval_metric="error", eval_set=eval_set, verbose=True)
                    clf.fit(train_x, train_y.T[i])

                    predict_test.append(clf.predict(test_x))

                predict_test = np.array(predict_test).T
                #print(predict_test)
                testoutput = testout_path + "\\"+ test_file_name +"_md_" + str(md)  + "lr_" + str(l)  + "nset_" + str(n) +'.csv' 
                errorout = testout_path + "\\"+ test_file_name +"md_" + str(md)  + "lr_" + str(l)  + "nset_" + str(n) +'_error.csv' 
                output(testoutput,(abs(predict_test)))
                #output(errorout,(abs(predict_test - test_y)/abs( test_y ))**2)

                score,squ_sum = getscore(test_y,predict_test)
                """
                if (score > max_score):
                    max_score = score
                    best_md = md
                    best_l = l
                    best_n = n
                """


    return squ_sum

            
            
            

In [58]:

train_paths=[]
test_paths=[]
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1v125c.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1vm40c.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9v125c.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9vm40c.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_tt1p0v25c.csv')

train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ff0p99v125c_base_400.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ff0p99vm40c_base_400.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ss0p81v125c_base_400.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_ss0p81vm40c_base_400.csv')
train_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\std\csv\lib1_tt0p9v25c_base_400.csv')

test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1vm40c_beta_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9v125c_beta_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9vm40c_beta_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_tt1p0v25c_beta_100.csv')

test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ff0p99v125c_alpha_100.csv')
test_paths.append( r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ff0p99vm40c_alpha_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ss0p81v125c_alpha_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_ss0p81vm40c_alpha_100.csv')
test_paths.append(r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\std\csv\lib1_tt0p9v25c_alpha_100.csv')

for i in range(len(test_paths)):
    print(train_paths[i])
    print(test_paths[i])




C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1v125c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1vm40c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1vm40c_beta_100.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9v125c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9v125c_beta_100.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ss0p9vm40c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ss0p9vm40c_beta_100.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_tt1p0v25c.csv
C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_

In [59]:
def delete_feature(data,feature):
    data.pop(feature)
    return data
    

In [60]:
squ_sum = 0.0
n = 0
for i in range(len(test_paths)):
    print( "train_file:" + train_paths[i] )
    print( "test_file:" + test_paths[i] )
    train,test = read_input( train_paths[i],test_paths[i] )
    
    
    
    
    #create new feature
    train.insert(1,'cell_trasition_power',train['cell_rise']*1+train['rise_transition']*2
                                            +train['cell_fall']*3+train['fall_transition']*4
                                            +train['rise_power']*5+train['fall_power']*6)
    test.insert(1,'cell_trasition_power',test['cell_rise']*1+test['rise_transition']*2
                                            +test['cell_fall']*3+test['fall_transition']*4
                                            +test['rise_power']*5+test['fall_power']*6)

    #delete minor feature
    train = delete_feature(train,"temperature")
    train = delete_feature(train,"voltage")
    train = delete_feature(train,"cell_rise")
    train = delete_feature(train,"rise_transition")
    train = delete_feature(train,"cell_fall")
    train = delete_feature(train,"fall_transition")
    train = delete_feature(train,"rise_power")
    train = delete_feature(train,"fall_power")
    train = delete_feature(train,"index1_1")
    train = delete_feature(train,"index1_2")
    train = delete_feature(train,"index1_3")
    train = delete_feature(train,"index1_4")
    train = delete_feature(train,"index1_5")
    train = delete_feature(train,"index1_6")
    train = delete_feature(train,"index1_7")
    train = delete_feature(train,"index2_1")
    
    
    
    test = delete_feature(test,"temperature")
    test = delete_feature(test,"voltage")
    test = delete_feature(test,"cell_rise")
    test = delete_feature(test,"rise_transition")
    test = delete_feature(test,"cell_fall")
    test = delete_feature(test,"fall_transition")
    test = delete_feature(test,"rise_power")
    test = delete_feature(test,"fall_power")    
    test = delete_feature(test,"index1_1")
    test = delete_feature(test,"index1_2")
    test = delete_feature(test,"index1_3")
    test = delete_feature(test,"index1_4")
    test = delete_feature(test,"index1_5")
    test = delete_feature(test,"index1_6")
    test = delete_feature(test,"index1_7")
    test = delete_feature(test,"index2_1")
    
    #print(train)
    #print(test)
    
    #df.groupby(['id'],as_index=False)['id'].agg({'cnt':'count'})
    
    
    
    for ctp in range(1,7):   
        print("cell_trasition_power:" + str(ctp))
        #train_x=train_x.sort_values(["cell_trasition_power"])
        train_1 = train[train['cell_trasition_power']==ctp]

        
        
        
        #test_x=test_x.sort_values(["cell_trasition_power"])    
        test_1 = test[test['cell_trasition_power']==ctp]
        
        #train_1 = train_1.drop_duplicates(subset=['capacitance'],keep='first',inplace=False)
        #test_1 = test_1.drop_duplicates(subset=['capacitance'],keep='first',inplace=False)
        
        #insert capacitance_total_cnt
        train_1=train_1.sort_values(["capacitance"])
        train_1_cnt =  train_1['capacitance'].value_counts(sort=False) 
        train_1_cnt = train_1_cnt.sort_index()
        train_1_cnt_dict = train_1_cnt.to_dict()
        train_1['capacitance_total_cnt']= train_1['capacitance'].map(train_1_cnt_dict)
        

        
        #insert capacitance_cnt_index
        train_1_cnt_list = train_1_cnt.to_list()
        train_1_cnt_list_insert = []
        for cnts in train_1_cnt_list:
            for cnt in range(cnts):
                train_1_cnt_list_insert.append(cnt+1)
                
        train_1.insert(1,'capacitance_cnt_index',train_1_cnt_list_insert)                
        #print(train_1)
        
        
        #insert capacitance_total_cnt
        test_1=test_1.sort_values(["capacitance"])
        test_1_cnt =  test_1['capacitance'].value_counts(sort=False) 
        test_1_cnt = test_1_cnt.sort_index()
        test_1_cnt_dict = test_1_cnt.to_dict()
        test_1['capacitance_total_cnt']= test_1['capacitance'].map(test_1_cnt_dict)
        

        
        #insert capacitance_cnt_index
        test_1_cnt_list = test_1_cnt.to_list()
        test_1_cnt_list_insert = []
        for cnts in test_1_cnt_list:
            for cnt in range(cnts):
                test_1_cnt_list_insert.append(cnt+1)
                
        test_1.insert(1,'capacitance_cnt_index',test_1_cnt_list_insert)                
        #print(test_1)
                
                
        
       
    
    
        #print(train_1)
        #print(test_1)
        train_y,test_y,train_x,test_x = spilt_data( train_1,test_1 )
    
        n = n + test_y.shape[0] * test_y.shape[1]
        print("n:" + str(n))
        #dataframe -> numpy.array.value
        train_x = train_x.values
        train_y = train_y.values
        test_x = test_x.values
        test_y = test_y.values
        testout_path = r'C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test_record'
        file,filesplit = os.path.split(test_paths[i])
        test_file_name = filesplit.split('.')[0]
        #print(test_file_name)
        squ_sum = squ_sum+xgb_train( train_x,train_y,test_x,test_y,testout_path ,test_file_name)


score = 100 - 100 * (np.sqrt(squ_sum/n))
print("total_score:" + str(score))



    
    

train_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1v125c.csv
test_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1v125c_beta_100.csv
df.shape : (18858, 74)
df.shape : (5658, 74)
cell_trasition_power:1
n:46060
md:6
lr:0.05
nest:500
accuracy:61.54768574565832
cell_trasition_power:2
n:92120
md:6
lr:0.05
nest:500
accuracy:72.20163664695437
cell_trasition_power:3
n:138180
md:6
lr:0.05
nest:500
accuracy:63.367515699083235
cell_trasition_power:4
n:184240
md:6
lr:0.05
nest:500
accuracy:68.46001710179013
cell_trasition_power:5
n:230741
md:6
lr:0.05
nest:500
accuracy:46.3605262063479
cell_trasition_power:6
n:277242
md:6
lr:0.05
nest:500
accuracy:57.63348137124839
train_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\train\boost\csv\train_ff1p1vm40c.csv
test_file:C:\Users\User\Fast_Timing_Model_Estimation_for_new_PVT-master\test\boost\csv\lib1_ff1p1vm40c_beta_100.csv
df.shape : (18858, 